# Regression and Other Stories: Logistic regression priors

Effect of priors in logistic regression. See Chapter 13 in Regression and Other Stories.

TODO: Revisit this notebook, I don't think it's working as expected

-----

## Load packages

In [1]:
using Turing, MCMCChains, GLM, Random, DataFrames
using StatsFuns: logistic

## Define a function running glm and a turing linear model with simulated data

Arguments are the number of simulated observations, and prior parameters a and b

In [2]:
## Turing Logistic Regression
@model function log_reg(x, y)
    # Linear
    α ~ Normal(0, 1)
    β ~ Normal(0, 1)
    
    μ = α .+ β * x
    p = logistic.(μ)
    y .~ Binomial.(1, p)
end;

# hand-rolled runif
function runif(n, lower, upper)
    return( rand(n) * (upper-lower) .+ lower )
end;


function bayes_sim(n, a=-2, b=0.8)
    
    println("Simulating with $n data points")
    
    x = runif(n, -1, 1)
    z = rand.(Logistic.( a .+ b*x ))
    y = ifelse.(z .> 0, 1, 0)
    
    df = DataFrame(Dict(
            "y"=>y,
            "x"=>x
            ))

    glm_fit = glm(@formula(y ~ x), df, Binomial(), LogitLink());

    miter = log_reg(x,y);
    chain = sample(miter, NUTS(), 4000);

    display(glm_fit);
    
    display(summarystats(chain));
    
end;

## Fit models to an increasing number of observations

In [3]:
Random.seed!(363852);

In [4]:
bayes_sim(10)

Simulating with 10 data points


┌ Info: Found initial step size
│   ϵ = 1.625
└ @ Turing.Inference C:\Users\tburch\.julia\packages\Turing\pwMQO\src\inference\hmc.jl:188
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling:   2%|██                                       |  ETA: 0:00:21┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:04

StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, LogitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x

Coefficients:
─────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error      z  Pr(>|z|)  Lower 95%  Upper 95%
─────────────────────────────────────────────────────────────────────────
(Intercept)  -1.29876     0.989711  -1.31    0.1894   -3.23856   0.641038
x             0.255941    1.8106     0.14    0.8876   -3.29276   3.80464
─────────────────────────────────────────────────────────────────────────

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat    ⋯
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64    ⋯

           α   -0.8922    0.6286     0.0099    0.0111   3351.6120    1.0003    ⋯
           β    0.2852    0.8433     0.0133    0.0129   3533.9382    1.0001    ⋯
                                                                1 column omitted


In [5]:
bayes_sim(100)

Simulating with 100 data points


┌ Info: Found initial step size
│   ϵ = 1.6
└ @ Turing.Inference C:\Users\tburch\.julia\packages\Turing\pwMQO\src\inference\hmc.jl:188
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling:   6%|███                                      |  ETA: 0:00:02┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:02


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, LogitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x

Coefficients:
────────────────────────────────────────────────────────────────────────
                Coef.  Std. Error      z  Pr(>|z|)  Lower 95%  Upper 95%
────────────────────────────────────────────────────────────────────────
(Intercept)  -3.85952    0.933976  -4.13    <1e-04   -5.69008   -2.02896
x             3.09491    1.33074    2.33    0.0200    0.48671    5.70311
────────────────────────────────────────────────────────────────────────

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat    ⋯
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64    ⋯

           α   -2.6111    0.3931     0.0062    0.0074   3149.4397    1.0002    ⋯
           β    1.1980    0.5776     0.0091    0.0104   2887.2504    0.9999    ⋯
                                                                1 column omitted


In [6]:
bayes_sim(1000)

Simulating with 1000 data points


┌ Info: Found initial step size
│   ϵ = 0.2
└ @ Turing.Inference C:\Users\tburch\.julia\packages\Turing\pwMQO\src\inference\hmc.jl:188
Sampling:   9%|████                                     |  ETA: 0:00:13┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
┌ Warning: The current proposal will be rejected due to numerical error(s).
│   isfinite.((θ, r, ℓπ, ℓκ)) = (true, false, false, false)
└ @ AdvancedHMC C:\Users\tburch\.julia\packages\AdvancedHMC\MIxdK\src\hamiltonian.jl:47
Sampling: 100%|█████████████████████████████████████████| Time: 0:00:13


StatsModels.TableRegressionModel{GeneralizedLinearModel{GLM.GlmResp{Vector{Float64}, Binomial{Float64}, LogitLink}, GLM.DensePredChol{Float64, LinearAlgebra.Cholesky{Float64, Matrix{Float64}}}}, Matrix{Float64}}

y ~ 1 + x

Coefficients:
──────────────────────────────────────────────────────────────────────────
                 Coef.  Std. Error       z  Pr(>|z|)  Lower 95%  Upper 95%
──────────────────────────────────────────────────────────────────────────
(Intercept)  -2.04456     0.105417  -19.39    <1e-83  -2.25118    -1.83795
x             0.993895    0.176493    5.63    <1e-07   0.647975    1.33982
──────────────────────────────────────────────────────────────────────────

Summary Statistics
  parameters      mean       std   naive_se      mcse         ess      rhat    ⋯
      Symbol   Float64   Float64    Float64   Float64     Float64   Float64    ⋯

           α   -2.0241    0.1018     0.0016    0.0019   3165.5021    1.0002    ⋯
           β    0.9535    0.1719     0.0027    0.0028   3341.9131    0.9999    ⋯
                                                                1 column omitted
